# Azure ML v2 — Batch Endpoint (1 endpoint, 3 deployments)

Creates/updates a batch endpoint and three deployments (`de`, `en`, `pl`).

In [ ]:
%pip -q install azure-ai-ml azure-identity python-dotenv

In [ ]:
import os
from pathlib import Path
from azure.identity import DefaultAzureCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import BatchEndpoint, BatchDeployment

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()


## Configure

In [ ]:
os.environ["SUBSCRIPTION_ID"] = os.getenv("SUBSCRIPTION_ID", "<sub-id>")
os.environ["RESOURCE_GROUP"] = os.getenv("RESOURCE_GROUP", "<rg>")
os.environ["WORKSPACE_NAME"] = os.getenv("WORKSPACE_NAME", "<ws>")
os.environ["ENDPOINT_NAME"] = os.getenv("ENDPOINT_NAME", "my-batch-ep")
os.environ["LOCATION"] = os.getenv("LOCATION", "westeurope")

os.environ["COMPUTE_DE"] = os.getenv("COMPUTE_DE", "cpu-de")
os.environ["COMPUTE_EN"] = os.getenv("COMPUTE_EN", "cpu-en")
os.environ["COMPUTE_PL"] = os.getenv("COMPUTE_PL", "cpu-pl")

os.environ["MODEL_NAME_DE"] = os.getenv("MODEL_NAME_DE", "clf-de")
os.environ["MODEL_NAME_EN"] = os.getenv("MODEL_NAME_EN", "clf-en")
os.environ["MODEL_NAME_PL"] = os.getenv("MODEL_NAME_PL", "clf-pl")


## Connect

In [ ]:
cred = DefaultAzureCredential(exclude_interactive_browser_credential=False)
ml_client = MLClient(
    cred,
    os.environ["SUBSCRIPTION_ID"],
    os.environ["RESOURCE_GROUP"],
    os.environ["WORKSPACE_NAME"],
)


## Upsert endpoint

In [ ]:
endpoint = BatchEndpoint(
    name=os.environ["ENDPOINT_NAME"],
    description="Batch endpoint with 3 deployments (de/en/pl)",
    auth_mode="aad_token",
    location=os.environ["LOCATION"],
)
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()
print("Endpoint upserted:", endpoint.name)


## Upsert deployments from YAML

In [ ]:
for p in [
    ROOT / "azureml" / "deployments" / "de.yml",
    ROOT / "azureml" / "deployments" / "en.yml",
    ROOT / "azureml" / "deployments" / "pl.yml",
]:
    dep = BatchDeployment.load(p)
    ml_client.batch_deployments.begin_create_or_update(dep).result()
    print("Upserted deployment:", dep.name)


## Submit a batch job (choose deployment)

In [ ]:
from azure.ai.ml import Input

input_path = "azureml://datastores/workspaceblobstore/paths/samples/input.jsonl"  # change me
job = ml_client.batch_endpoints.invoke(
    endpoint_name=os.environ["ENDPOINT_NAME"],
    deployment_name="en",  # de|en|pl
    input=Input(type="uri_file", path=input_path),
)
print("Submitted job:", job.name)
